In [1]:
import opendssdirect as dss
import os
import pathlib
import numpy as np
import pandas as pd
import random
import sys
from scipy.stats import *
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import seaborn as sns
import math
from matplotlib.animation import FuncAnimation
from celluloid import Camera

In [2]:
dss.__version__
dss.run_command('Redirect C:/Users/smartgrid_AI/Documents/kms/Hyundai/IEEE34BUSSYSTEM/ieee34Mod2.dss')
dss.run_command("New Energymeter.M1  Line.L1  1")
dss.Solution.Solve()

In [3]:
Ymatrixori = dss.Circuit.SystemY()
Ymatrix = np.array(Ymatrixori).reshape(138,-1)
Gm = Ymatrix[:, 0::2]
Bm = Ymatrix[:, 1::2]
Gm.shape

(138, 138)

In [4]:
dss.run_command("export Voltages")
Vthetmatrix = pd.read_csv("ieee34-2_EXP_VOLTAGES.csv", encoding='CP949', error_bad_lines=False)

In [5]:
def VTextract(Vthetmatrix):
    Vmat = np.zeros((Vthetmatrix.shape[0],3))
    Tmat = np.zeros((Vthetmatrix.shape[0],3))
    dis = np.zeros((Vthetmatrix.shape[0],3), dtype=int)
    for i in range(Vthetmatrix.shape[0]):
        for j in range(3):
            Vmat[i,j] = Vthetmatrix.iloc[i,4*j+3]*np.sqrt(3)
            Tmat[i,j] = Vthetmatrix.iloc[i,4*j+4]*np.pi/180
            if Vmat[i,j]!=0:
                dis[i,j]=1
    Vm = np.zeros(np.sum(dis))
    Tm = np.zeros(np.sum(dis))
    ena = 0
    for a in range(Vthetmatrix.shape[0]):
        for b in range(3):
            if dis[a,b]==1:
                Vm[ena]=Vmat[a,b]
                Tm[ena]=Tmat[a,b]
                ena = ena+1             
    return ena, Vm, Tm

def Sensecalc(ena, Vm, Tm):
    j1 = np.zeros((ena,ena))
    j2 = np.zeros((ena,ena))
    j3 = np.zeros((ena,ena))
    j4 = np.zeros((ena,ena))

    for a in range(ena):
        for b in range(ena):
            if a!=b:
                j1[a,b]=Vm[a]*Vm[b]*(Gm[a,b]*np.sin(Tm[a]-Tm[b])-Bm[a,b]*np.cos(Tm[a]-Tm[b]))
                j2[a,b]=Vm[a]*(Gm[a,b]*np.cos(Tm[a]-Tm[b])+Bm[a,b]*np.sin(Tm[a]-Tm[b]))
                j3[a,b]=-Vm[a]*Vm[b]*(Gm[a,b]*np.cos(Tm[a]-Tm[b])+Bm[a,b]*np.sin(Tm[a]-Tm[b]))
                j4[a,b]=Vm[a]*(Gm[a,b]*np.sin(Tm[a]-Tm[b])-Bm[a,b]*np.cos(Tm[a]-Tm[b]))
            if a==b:
                for c in range(ena):
                    if c!=a:
                        j1[a,b]=j1[a,b]-Vm[a]*Vm[c]*(Gm[a,c]*np.sin(Tm[a]-Tm[c])-Bm[a,c]*np.cos(Tm[a]-Tm[c]))
                        j2[a,b]=j2[a,b]+Vm[c]*(Gm[a,c]*np.cos(Tm[a]-Tm[c])+Bm[a,c]*np.sin(Tm[a]-Tm[c]))
                        j3[a,b]=j3[a,b]+Vm[a]*Vm[c]*(Gm[a,c]*np.cos(Tm[a]-Tm[c])+Bm[a,c]*np.sin(Tm[a]-Tm[c]))
                        j4[a,b]=j4[a,b]+Vm[c]*(Gm[a,c]*np.sin(Tm[a]-Tm[c])-Bm[a,c]*np.cos(Tm[a]-Tm[c]))
                    if c==a:
                        j2[a,b]=j2[a,b]+2*Vm[a]*Gm[a,b]
                        j4[a,b]=j4[a,b]-2*Vm[a]*Bm[a,b]                 
            if a<3 and b<3:
                j1[a,b]=0
                j2[a,b]=0
                j3[a,b]=0
                j4[a,b]=0      

    svq=np.linalg.inv(j4-np.matmul(j3,np.matmul(np.linalg.pinv(j1),j2)))
    svp=np.linalg.inv(j2-np.matmul(j1,np.matmul(np.linalg.pinv(j3),j4)))
    return svp, svq

In [6]:
ena, Vm, Tm = VTextract(Vthetmatrix)
Vmini = Vm
svp, svq = Sensecalc(ena, Vm, Tm)

In [7]:
pd.DataFrame(svq)

,0,1,2,3,4,5,6,7,8,9,...,128,129,130,131,132,133,134,135,136,137
0,2.217744e+07,7.755761e+07,-9.973500e+07,-3.873222e-08,-3.642068e-08,3.367107e-09,-2.350192e-05,2.444614e-05,3.519778e-06,-2.069900e-05,...,-1.498859e-06,-5.058739e-06,-4.755213e-06,-1.490151e-06,-5.129388e-06,-1.532859e-06,-1.546070e-06,7.209830e-07,-4.478862e-07,-9.790964e-06
1,-4.970843e+07,9.764876e+07,-4.794045e+07,-9.938780e-10,-7.400525e-09,-1.977130e-08,-1.181388e-05,8.435282e-06,7.990934e-07,-9.520515e-06,...,-1.130826e-05,-4.209568e-06,-3.394879e-06,-1.148946e-05,-4.240226e-06,-1.165122e-05,-1.174892e-05,-1.093368e-05,8.140031e-06,-3.099574e-06
2,2.753099e+07,-1.752064e+08,1.476754e+08,1.344012e-08,1.753552e-08,-9.881584e-09,3.528962e-05,-3.290761e-05,-4.345072e-06,3.019333e-05,...,1.278529e-05,9.245117e-06,8.127654e-06,1.295777e-05,9.346425e-06,1.316225e-05,1.327316e-05,1.019239e-05,-7.711441e-06,1.286963e-05
3,-1.517545e-08,1.891250e-08,-1.322285e-08,3.161936e-09,-4.319254e-09,1.157340e-09,3.148717e-09,-4.323004e-09,1.175357e-09,3.148793e-09,...,-4.722081e-09,2.226530e-09,2.663569e-09,-4.722341e-09,2.226646e-09,-4.722486e-09,-4.722492e-09,2.830467e-09,-5.227940e-09,2.616349e-09
4,-2.494047e-09,-8.273338e-09,1.281664e-09,1.157342e-09,3.161906e-09,-4.319240e-09,1.175952e-09,3.148367e-09,-4.323705e-09,1.179166e-09,...,2.493879e-09,-4.840808e-09,2.272907e-09,2.493935e-09,-4.840605e-09,2.493922e-09,2.493925e-09,2.655174e-09,2.607438e-09,-5.376374e-09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,-1.841651e-06,5.165016e-07,1.315840e-06,1.537593e-09,3.014532e-09,-4.552114e-09,-9.687739e-08,2.474343e-05,-9.185055e-06,-1.401197e-07,...,1.824808e-03,-7.841839e-04,3.838928e-05,1.841935e-03,-7.896024e-04,1.867473e-03,1.867476e-03,1.103472e-04,1.637863e-03,-9.116444e-04
134,-1.841634e-06,5.119637e-07,1.320361e-06,1.537595e-09,3.014536e-09,-4.552120e-09,-9.687751e-08,2.474346e-05,-9.185067e-06,-1.401199e-07,...,1.824810e-03,-7.841850e-04,3.838933e-05,1.841937e-03,-7.896034e-04,1.867476e-03,1.893022e-03,1.103474e-04,1.637865e-03,-9.116456e-04
135,-2.995209e-07,7.393181e-07,-4.412030e-07,5.399676e-10,-7.950703e-10,2.551063e-10,4.055122e-06,-1.886383e-06,3.816363e-08,5.422434e-06,...,-1.515440e-04,1.691135e-05,2.917265e-04,-1.515578e-04,1.692125e-05,-1.515640e-04,-1.515642e-04,1.278966e-03,-4.604222e-04,-8.931814e-06
136,-2.678332e-07,4.671048e-08,2.197100e-07,2.210986e-10,5.351055e-10,-7.562027e-10,-1.015821e-07,4.086783e-06,-1.639951e-06,-1.374931e-07,...,2.799035e-04,-1.335287e-04,3.721504e-06,2.799079e-04,-1.335229e-04,2.799083e-04,2.799087e-04,-5.190674e-05,1.264848e-03,-4.044971e-04


In [8]:
allnodename = dss.Circuit.AllNodeNames()
allnode = np.array(allnodename)

In [9]:
def USERPV():
    try:
        pv_list = list(map(int, input('PV 모선 번호를 입력하세요: ').split()))
        pv_array = np.array(pv_list, dtype = int)
        num_pv = pv_array.shape[0]
        pv_busind = np.zeros(num_pv, dtype=int)
        nodelist = []
    
        for i in range(num_pv):
            for j in range(1,4):
                try:
                    x = np.where(allnode == str(pv_array[i])+'.'+str(j))[0][0]
                    nodelist.append(x)
                except:
                    print(str(pv_array[i])+"번 모선은 단상 모선입니다")
                    USERPV()
    except:
        print("PV 모선 번호가 잘못 입력되었습니다")
        USERPV()
    return pv_array, num_pv, np.array(nodelist, dtype=int)     

In [10]:
## example value: 890 848 840 832 828

pv_array, num_pv, pvnodeind = USERPV()

PV 모선 번호를 입력하세요:  890 848 840 832 828


In [11]:
pv_array, num_pv, pvnodeind

(array([890, 848, 840, 832, 828]),
 5,
 array([135, 136, 137, 105, 106, 107,  87,  88,  89,  63,  64,  65,  51,
         52,  53]))

In [12]:
svp_pv = svp[:, pvnodeind]
svp_pv

array([[-2.73022264e-05,  4.43472051e-05, -4.60988838e-05, ...,
        -5.53648991e-05,  3.30926802e-05, -1.88155264e-05],
       [-6.99445929e-06,  7.71286518e-06, -9.34020610e-06, ...,
        -9.46754401e-06,  6.74660988e-06, -3.64586275e-06],
       [ 3.42642597e-05, -5.20900145e-05,  5.54076138e-05, ...,
         6.48030767e-05, -3.98670539e-05,  2.24328902e-05],
       ...,
       [ 1.25330575e-03,  2.93923022e-04, -5.12584480e-04, ...,
         2.27138451e-04,  5.02195762e-05, -1.01431070e-04],
       [-5.44646231e-04,  1.21572673e-03,  2.53785794e-04, ...,
        -1.05712774e-04,  2.05120358e-04,  4.26039481e-05],
       [ 2.76039834e-04, -4.77923530e-04,  1.23432560e-03, ...,
         4.67649736e-05, -9.13698677e-05,  2.10105274e-04]])

In [13]:
popn = 300
MaxIt = 500
hUB = np.ones(num_pv)*500
LBi = np.zeros(num_pv)
uv = 1.05
lv = 0.95
It = 1

In [14]:
def checkvoltage(Vm):
    V2pu = np.array(Vm)
    for i in range(len(V2pu)):
        if V2pu[i]>60000:
            V2pu[i] = Vm[i]/69000
        elif V2pu[i]>20000:
            V2pu[i] = Vm[i]/24900
        else:
            V2pu[i] = Vm[i]/4160

    V2vio_u = (V2pu[3:]-uv>0)
    V2vio_l = (V2pu[3:]-lv<0)
    vio_node = V2vio_u+V2vio_l
    return vio_node, V2pu

def rk_initialization(popn, dim, lb, ub):
    x = np.zeros((popn, dim), dtype=int)
    for i in range(dim):
        try:
            x[:,i] = np.random.randint(lb[i], ub[i], size=(popn))
        except:
            x[:,i] = 0
    return x

def RungeKutta(Xb, Xw, DelX):
    dim = Xb.shape[1]
    C = (np.random.randint(2)+1)*(1-np.random.rand(1))
    r1 = np.random.rand(1, dim)
    r2 = np.random.rand(1, dim)
    
    K1 = 0.5*(np.random.rand(1)*Xw - C*Xb)
    K2 = 0.5*(np.random.rand(1)*(Xw+r2*np.multiply(K1, DelX)/2) - (C*Xb+r1*np.multiply(K1, DelX)/2))
    K3 = 0.5*(np.random.rand(1)*(Xw+r2*np.multiply(K2, DelX)/2) - (C*Xb+r1*np.multiply(K2, DelX)/2))
    K4 = 0.5*(np.random.rand(1)*(Xw+r2*np.multiply(K3, DelX)) - (C*Xb+r1*np.multiply(K3, DelX)))
    
    XRK = (K1+2*K2+2*K3+K4)
    SM = XRK/6
    return SM

def objfunction(xbest, delx, uv, lv, svp_pv, Vm):
    x = xbest+delx
    # function 1: Sum of PV active power
    f1 = -sum(x[0,:])
    
    # function 2: Violation of Voltage
    PVpnode = np.zeros(3*(x.shape[1]), dtype=float)
    for i in range(x.shape[1]):
        PVpnode[3*i:3*i+3] = delx[0,i]/3
    #print(x.shape,PVpnode.shape)
    Vnew = Vm+1000*np.matmul(svp_pv,PVpnode)
    _,V2pu = checkvoltage(Vnew)

    V2vio_u = (V2pu[3:]-uv>0)
    V2vio_l = (V2pu[3:]-lv<0)
    f2 = 1e5*sum(V2vio_u) + 1e5*sum(V2vio_l)
    return f1+f2

def RndX(popn,i):
    Qi = np.random.permutation(popn)
    Qi = np.delete(Qi, np.where(Qi==i)[0][0])
    A = Qi[0]
    B = Qi[1]
    C = Qi[2]
    return A, B, C

def Unifrnd(a,b,c,dim):
    a2 = a/2
    b2 = b/2
    mu = a2+b2
    sig = b2-a2
    z = mu+sig*(2*np.random.rand(c,dim)-1)
    return z

In [15]:
def PVratechange(allPV_name, Best_A):
    k=0
    dss.PVsystems.First()
    for pvname in allPV_name:     
        dss.run_command("PVSystem."+pvname+".kVA="+str(Best_A[k]))
        dss.run_command("PVSystem."+pvname+".Pmpp="+str(Best_A[k]))
        #print("PVSystem."+pvname+".Pmpp="+str(Best_A[k]))
        k = k+1
        if not dss.PVsystems.Next() > 0:
            break

def Boundaryupdate(Best_X):
    LB = -Best_X
    UB = hUB-Best_X
    return LB, UB

def rcost_check2(Best_A, X, best_cost, Vm):
    Best_Aa = Best_A + X
    LB, UB = Boundaryupdate(Best_Aa)
    allPV_name = dss.PVsystems.AllNames()
    PVratechange(allPV_name, Best_Aa)
    dss.Solution.Solve()
    dss.run_command("export Voltages")
    Vthetmatrix = pd.read_csv("ieee34-2_EXP_VOLTAGES.csv", encoding='CP949', error_bad_lines=False)
    ena, Vm, Tm = VTextract(Vthetmatrix)
    svp, svq = Sensecalc(ena, Vm, Tm)
    svp_pv = svp[:, pvnodeind]
    rcost = objfunction(Best_Aa, np.zeros(num_pv).reshape(1,-1), uv, lv, svp_pv, Vm)
    return rcost, svp_pv, Vm, LB, UB

def run_RK(X, Best_X, Best_A, best_ind, best_cost, MaxIt, Xnew, conv_curve, cost, max_indure, LB, UB, svp_pv, Vm):
    update_check = 0
    for It in range(MaxIt):
        It = It+1
        f = 20*np.exp(-12*It/MaxIt)
        Xavg = np.mean(X, axis=0)
        SF = 2*0.5-np.random.rand(1,popn)*f

        for i in range(popn):
            l_cost = np.min(cost)
            l_ind = np.argmin(cost)
            lbest = X[l_ind,:].reshape(1,-1)
            A, B, C = RndX(popn, i)
            ind1 = np.argmin(cost[[A,B,C]])
            
            #print(X.shape, Best_X.shape, Xavg.shape)
            gamma = np.random.rand(1)*(X[i,:]-np.random.rand(1,X.shape[1])*(UB-LB))*np.exp(-4*It/MaxIt)
            Stp = np.random.rand(1,X.shape[1])*((Best_X - np.random.rand(1)*Xavg))
            DelX = 2*np.multiply(np.random.rand(1,X.shape[1]),np.abs(Stp))

            if cost[i]<cost[ind1]:
                Xb = X[i,:].reshape(1,-1)
                Xw = X[ind1,:].reshape(1,-1)
            else:
                Xb = X[ind1,:].reshape(1,-1)
                Xw = X[i,:].reshape(1,-1)

            SM = RungeKutta(Xb, Xw, DelX)

            L = np.random.rand(1,X.shape[1])<0.5
            Xc = np.multiply(L, X[i,:])+np.multiply(1-L,X[A,:])
            Xm = np.multiply(L, Best_X)+np.multiply(1-L,lbest)

            vec = np.array((1, -1))
            flag = np.array(np.floor(2*np.random.rand(1,X.shape[1])), dtype=int)

            r = vec[flag]

            g = 2*np.random.rand(1)
            mu = 0.5+np.random.randn(1,X.shape[1])*0.1

            if np.random.rand(1)<0.5:
                Xnew = (Xc + r*SF[0,i]*g*Xc) + SF[0,i]*SM + mu*(Xm-Xc)
            else:
                Xnew = (Xm + r*SF[0,i]*g*Xm) + SF[0,i]*SM + mu*(X[A,:]-X[B,:])

            FU = Xnew>UB
            FL = Xnew<LB
            Xnew = np.multiply(Xnew, ~(FU+FL)) + np.multiply(UB, FU) + np.multiply(LB, FL)
            CostNew = objfunction(Best_A, Xnew, uv, lv, svp_pv, Vm)
            if CostNew<cost[i]:
                X[i,:] = Xnew
                cost[i] = CostNew

            if np.random.rand(1)<0.5:
                EXP = np.exp(-5*np.random.rand(1)*It/MaxIt)
                r = np.floor(Unifrnd(-1,2,1,1))
                u = 2*np.random.rand(1,X.shape[1])
                w = Unifrnd(0,2,1,X.shape[1])*EXP

                A, B, C = RndX(popn, i)
                Xavg = (X[A,:]+X[B,:]+X[C,:])/3

                beta = np.random.rand(1,X.shape[1])
                Xnew1 = beta*Best_X+(1-beta)*Xavg
                Xnew2 = np.zeros(Xnew1.shape)
                for j in range(X.shape[1]):
                    if w[0,j]<1:
                        Xnew2[0,j] = Xnew1[0,j]+r*w[0,j]*np.abs((Xnew1[0,j]-Xavg[j])+np.random.randn(1))
                    else:
                        Xnew2[0,j] = Xnew1[0,j]-Xavg[j]+r*w[0,j]*np.abs(np.multiply(u[0,j], Xnew1[0,j])-Xavg[j]+np.random.randn(1))

                FU = Xnew2>UB
                FL = Xnew2<LB
                
                Xnew2 = np.multiply(Xnew2, ~(FU+FL)) + np.multiply(UB, FU) + np.multiply(LB, FL)
                CostNew = objfunction(Best_A, Xnew2, uv, lv, svp_pv, Vm)

                if CostNew<cost[i]:
                    X[i,:] = Xnew2
                    cost[i] = CostNew
                else:
                    if np.random.rand(1)<w[0,np.random.randint(X.shape[1])]:
                        SM = RungeKutta(X[i,:].reshape(1,-1), Xnew2, DelX)
                        Xnew = (Xnew2-np.random.rand(1)*Xnew2) + SF[0,i]*(SM+2*np.multiply(np.random.rand(1,X.shape[1]),Best_X)-Xnew2)
                        FU = Xnew>UB
                        FL = Xnew<LB
                        Xnew = np.multiply(Xnew, ~(FU+FL)) + np.multiply(UB, FU) + np.multiply(LB, FL)
                        CostNew = objfunction(Best_A, Xnew, uv, lv, svp_pv, Vm)

                        if CostNew<cost[i]:
                            X[i,:] = Xnew
                            cost[i] = CostNew

                if cost[i]<best_cost:
                    rcost, svp_pv_im, Vm_im, LB_im, UB_im = rcost_check2(Best_A, X[i,:], best_cost, Vm)
                    cost[i]=rcost
                    if rcost<best_cost:
                        print("The best cost changes from "+str(best_cost)+" to "+str(rcost)+" with positions: "+str(Best_A+X[i,:]))
                        svp_pv = svp_pv_im
                        Best_A = Best_A + X[i,:]
                        Best_X = np.zeros(5)
                        best_cost = rcost
                        Vm = Vm_im
                        if rcost<0:
                            LB = -Best_A
                            UB = np.ones(5)*50
                        else:
                            LB = -Best_A
                            UB = hUB-Best_A

                        print(LB, UB)
                        X = rk_initialization(popn, num_pv, LB, UB)
                        X[0,:] = 0
                        for i in range(popn):
                            cost[i] = objfunction(Best_A, X[i,:].reshape(1,-1), uv, lv, svp_pv, Vm)
                        Xavg = np.mean(X, axis=0)
                        
        conv_curve[0,It] = best_cost

        if conv_curve[0,It]>=conv_curve[0,It-1]:
            update_check = update_check+1
            if update_check > max_indure:
                break        
        else:
            update_check = 0

        if It%10 == 0:
            print("At iteration "+str(It)+", the best cost is "+str(best_cost)+" with the positions: "+str(Best_A))
            
    allPV_name = dss.PVsystems.AllNames()        
    PVratechange(allPV_name, Best_A)
    print("Simulation End")
    return X, Best_X, Best_A, best_ind, best_cost, conv_curve, cost, max_indure, LB, UB, svp_pv, Vm

In [16]:
for go in range(10):
    ### Find Initial Best Position
    Acost = np.zeros((popn,1))

    A = rk_initialization(popn, num_pv, LBi, hUB)

    A[0,:] = 0

    for i in range(popn):
        Acost[i] = objfunction(np.zeros(num_pv), A[i,:].reshape(1,-1), uv, lv, svp_pv, Vmini)

    best_Acost = np.min(Acost)
    best_Aind = np.argmin(Acost)
    Best_A = A[best_Aind,:]

    print(best_Acost, Best_A)

    dss.run_command('Redirect C:/Users/smartgrid_AI/Documents/kms/Hyundai/IEEE34BUSSYSTEM/ieee34Mod2.dss')
    dss.run_command("New Energymeter.M1  Line.L1  1")
    for m in range(0,num_pv):
        if Best_A[m]!=0:
            if Vm[pvnodeind[3*m]]>50000:
                basekV = 69
            elif Vm[pvnodeind[3*m]]>15000:
                basekV = 24.9
            else:
                basekV = 4.16
            pv_kva_text="New PVSystem.PV"+str(m+1)+" conn=Wye phases=3 bus="+str(pv_array[m])+" kV="+str(basekV)+" irrad=1.0 Pmpp="+str(Best_A[m])+" temperature=25 kVA="+str(Best_A[m])+" WattPriority=True"
            print(pv_kva_text)
            dss.run_command(pv_kva_text)

    dss.Solution.Solve()
    dss.run_command("export Voltages")
    Vthetmatrix = pd.read_csv("ieee34-2_EXP_VOLTAGES.csv", encoding='CP949', error_bad_lines=False)
    ena, Vm, Tm = VTextract(Vthetmatrix)
    svp, svq = Sensecalc(ena, Vm, Tm)
    svp_pv = svp[:, pvnodeind]
    Vviol,V2pu = checkvoltage(Vm)

    LB, UB = Boundaryupdate(Best_A)
    X = rk_initialization(popn, num_pv, LB, UB)
    X[0,:] = 0

    cost = np.zeros((popn,1))
    conv_curve = np.zeros((1, MaxIt+1))
    for i in range(popn):
        cost[i] = objfunction(Best_A, X[i,:].reshape(1,-1), uv, lv, svp_pv, Vm)

    Xnew = np.zeros((1, num_pv))    
    best_cost = np.min(cost)
    best_ind = np.argmin(cost)
    Best_X = X[best_ind,:]
    conv_curve[0] = best_cost

    max_indure = 30
    X, Best_X, Best_A, best_ind, best_cost, conv_curve, cost, max_indure, LB, UB, svp_pv, Vm=run_RK(X, Best_X, Best_A, best_ind, best_cost, MaxIt, Xnew, conv_curve, cost, max_indure, LB, UB, svp_pv, Vm)

    rcost, _, Vm1best, _, _ =rcost_check2(Best_A, Best_X, best_cost, Vmini)
    vio_node, V2pu = checkvoltage(Vm1best)
    
    if rcost<0:
        break
        
if rcost<0:
    print("The Biggest PV Output is "+str(-rcost)+" with PV size "+str(Best_A)+" for bus "+str(pv_array))
else:
    print("PV cannot be installed because of voltage problem")

400000.0 [0 0 0 0 0]
The best cost changes from 400000.0 to 399634.0 with positions: [183   0 183   0   0]
[-183    0 -183    0    0] [317. 500. 317. 500. 500.]
The best cost changes from 399634.0 to 399200.0 with positions: [ 86 201 210 130 173]
[ -86 -201 -210 -130 -173] [414. 299. 290. 370. 327.]
The best cost changes from 399200.0 to 399002.0 with positions: [ 59 420 190 232  97]
[ -59 -420 -190 -232  -97] [441.  80. 310. 268. 403.]
The best cost changes from 399002.0 to 398500.0 with positions: [500   0 500   0 500]
[-500    0 -500    0 -500] [  0. 500.   0. 500.   0.]
The best cost changes from 398500.0 to 398279.0 with positions: [500   0 500 221 500]
[-500    0 -500 -221 -500] [  0. 500.   0. 279.   0.]
The best cost changes from 398279.0 to 398036.0 with positions: [500 370 500 198 396]
[-500 -370 -500 -198 -396] [  0. 130.   0. 302. 104.]
The best cost changes from 398036.0 to 397620.0 with positions: [500 500 500 380 500]
[-500 -500 -500 -380 -500] [  0.   0.   0. 120.   0.]